In [141]:
import pandas as pd
from arcgis.features import GeoAccessor, GeoSeriesAccessor
import os
from datetime import datetime
import arcpy

arcpy.env.overwriteOutput = True

In [142]:
sheet_name = 'Estandarizacion_Asignaciones'
sheet_id = '1a6AHnpCTLWokb12ZhfjGp3QTkmfQoQ9DKbDARd35LXo'

url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"

tipo_datos = {'NUMERO_PRE': str,
    'CODIGO_TER':str,
    'CODIGO_MUN': str,
    'MATRICULA_': str
    }

asignacion_reconocimiento = pd.read_csv(url, dtype=tipo_datos)
columnas_seleccion = [
    'NUMERO_PRE', 
    'CODIGO_TER',
    'CODIGO_MUN', 
    'MATRICULA_',
    'TOTAL_REGI',
    'TIPO_DOCUM',
    'NUMERO_DOC',
    'NOMBRE',
    'DIRECCION',
    'DESTINO_EC',
    'Area_ha_cm',
    'NOMBRE_VER',
    'METODO',
    'SEMANA',
    'COORD',
    'RECON',
    'NOMBRE_ARCHIVO_SOLICITUD',
    ]

fecha_actual = datetime.now()
fecha_espacializacion = str(fecha_actual.strftime("%Y-%m-%d"))

RUTA_FILE_GDB_CONSOLIDADO = r"C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\E1_Alistamiento_Diagnostico\3_Disposicion\1.BD_Consolidada\BD_Consolidada_Lote4.gdb"
RUTA_FILE_GDB_ASIGNACION = r"C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\zApoyo_A_Equipos\zApoyoEquipoReconocimiento\2.Asignacion_Vs_ObservacionesJuridicas\Base_Datos\Asignacion_Reconocimiento.gdb"

RUTA_XLSX_ASIGNACION_OBSJURIDICAS = r"C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\zApoyo_A_Equipos\zApoyoEquipoReconocimiento\2.Asignacion_Vs_ObservacionesJuridicas\Estadisticos"
NOMBRE_XLSX_ASIGNACION_OBSJURIDICAS = 'asignacion_obsjuridicas.xlsx'

CAPA_TERRENOS_R = 'Analitica_UT_Consolidada\R_TERRENO'
CAPA_TERRENOS_U = 'Analitica_UT_Consolidada\\U_TERRENO'

DATASET_ANALITICA = 'Analitica_UT_Consolidada'
CAPA_OBS_JURIDICA = 'TERRENOS_CON_OBSERVACIONES_JURIDICAS'

COLUMNAS_REPORTE_XLS_ASIGNACION_OBSJURIFICAS = [
    'NUMERO_PRE', 
    'CODIGO_MUN', 
    'MATRICULA_',
    'TOTAL_REGI',
    'TIPO_DOCUM',
    'NUMERO_DOC',
    'NOMBRE',
    'DIRECCION',
    'DESTINO_EC',
    'Area_ha_cm',
    'NOMBRE_VER',
    'METODO',
    'SEMANA',
    'COORD',
    'RECON',
    'NOMBRE_ARCHIVO_SOLICITUD',
    'MATRICULA_INMOBILIARIA',
    'ÁREA_FMI',
    'NRO_ANOTACIÓN_FMI',
    'OBSERVACIONES']

RENOMBRE_COLUMNAS_REPORTE_XLS_ASIGNACION_OBSJURIDICAS = {
    'NUMERO_PRE':'asignacion_npn', 
    'CODIGO_MUN':'asignacion_codigomunicipio', 
    'MATRICULA_':'asignacion_fmi',
    'TOTAL_REGI':'asignacion_totalregistros',
    'TIPO_DOCUM':'asignacion_tipodocumento',
    'NUMERO_DOC':'asignacion_numerodocumento',
    'NOMBRE':'asignacion_nombre',
    'DIRECCION':'asignacion_direccion',
    'DESTINO_EC':'asignacion_destinacioneconomica',
    'Area_ha_cm':'asignacion_areahactm12',
    'NOMBRE_VER':'asignacion_nombrevereda',
    'METODO':'asignacion_metodo',
    'SEMANA':'asignacion_semana',
    'COORD':'asignacion_coordinador',
    'RECON':'asignacion_reconocedor',
    'NOMBRE_ARCHIVO_SOLICITUD':'asignacion_nombrearchivosolicitud',
    'MATRICULA_INMOBILIARIA':'obsjuridica_fmi',
    'ÁREA_FMI':'obsjuridica_areafmi',
    'NRO_ANOTACIÓN_FMI':'obsjuridica_numeroanotacionfmi',
    'OBSERVACIONES':'obsjuridica_observaciones'
}

In [143]:
asignacion_reconocimiento_filtro = asignacion_reconocimiento[columnas_seleccion]

asignacion_reconocimiento_filtro = asignacion_reconocimiento_filtro.copy()

print(asignacion_reconocimiento_filtro.shape[0])

690


In [144]:
print(asignacion_reconocimiento_filtro['CODIGO_MUN'].value_counts())

asignacion_reconocimiento_filtro.loc[asignacion_reconocimiento_filtro['CODIGO_MUN'] == '8606', 'CODIGO_MUN'] = asignacion_reconocimiento_filtro['NUMERO_PRE'].str[:5]
asignacion_reconocimiento_filtro.loc[asignacion_reconocimiento_filtro['CODIGO_MUN'].isna(), 'CODIGO_MUN'] = asignacion_reconocimiento_filtro['NUMERO_PRE'].str[:5]

print(asignacion_reconocimiento_filtro['CODIGO_MUN'].value_counts())

CODIGO_MUN
8606    493
Name: count, dtype: int64
CODIGO_MUN
08606    690
Name: count, dtype: int64


In [145]:
asignacion_reconocimiento_filtro['FECHA_ESPACIALIZACION'] = fecha_espacializacion

In [146]:

capa_terrenos_rurales = os.path.join(RUTA_FILE_GDB_CONSOLIDADO, CAPA_TERRENOS_R)
capa_terrenos_urbanos = os.path.join(RUTA_FILE_GDB_CONSOLIDADO, CAPA_TERRENOS_U)

# ** Terrenos Rurales
df_terreno_rural = pd.DataFrame.spatial.from_featureclass(capa_terrenos_rurales)
df_terreno_rural.head()
# ** Terrenos Urbanos
df_terreno_urbano = pd.DataFrame.spatial.from_featureclass(capa_terrenos_urbanos)

# ** Selección de columnas
columnas_selecciones = ['CODIGO',
                        'CODIGO_ANTERIOR',
                        'SHAPE']

# ** DF Terrenos Rurales, Urbanos
df_terreno_rural_reducido = df_terreno_rural[columnas_selecciones]
df_terreno_urbano_reducido = df_terreno_urbano[columnas_selecciones]

# ** Concatenación de dataframes
df_terrenos = pd.concat([df_terreno_rural_reducido, df_terreno_urbano_reducido])

In [147]:
# ** Terrenos Gestionados SIG Espacializados
df_terrenos_asignacion_reconocimiento = pd.merge(df_terrenos, asignacion_reconocimiento_filtro, left_on='CODIGO', right_on='NUMERO_PRE', how='inner')

In [148]:

# ** Incorporacion a BD Consolidada
NOMBRE_CAPA_SALIDA = f"Analitica_UT_Consolidada\\{'terrenos_con_control_de_cambios_esig'.upper()}"
ruta_salida_asignacion_reconocimiento_bdconsolidada = os.path.join(RUTA_FILE_GDB_CONSOLIDADO, NOMBRE_CAPA_SALIDA)

df_terrenos_asignacion_reconocimiento.spatial.to_featureclass(location=ruta_salida_asignacion_reconocimiento_bdconsolidada)
print(f"Se exporta la capa {NOMBRE_CAPA_SALIDA}")

# ** Incorporacion a BD Asignación
NOMBRE_CAPA_SALIDA = f"{'Asignacion_Reconocimiento'.upper()}"
ruta_salida_asignacion_reconocimiento_bdasignada = os.path.join(RUTA_FILE_GDB_ASIGNACION, NOMBRE_CAPA_SALIDA)

df_terrenos_asignacion_reconocimiento.spatial.to_featureclass(location=ruta_salida_asignacion_reconocimiento_bdasignada)
print(f"Se exporta la capa {NOMBRE_CAPA_SALIDA}")

Se exporta la capa Analitica_UT_Consolidada\TERRENOS_CON_CONTROL_DE_CAMBIOS_ESIG
Se exporta la capa ASIGNACION_RECONOCIMIENTO


<h3>Observaciones Juridicas</h3>

In [149]:
arcpy.env.workspace = RUTA_FILE_GDB_CONSOLIDADO

for dataset in arcpy.ListDatasets():
    if dataset == DATASET_ANALITICA:
        for capa in arcpy.ListFeatureClasses(feature_dataset=dataset):
            if capa == CAPA_OBS_JURIDICA:
                capa_observaciones_juricas = os.path.join(arcpy.env.workspace, dataset, capa)
                df_observaciones_juridicas = pd.DataFrame.spatial.from_featureclass(capa_observaciones_juricas)
                print(f"Se crea DF de capa {capa}, en dataset {dataset}")

Se crea DF de capa TERRENOS_CON_OBSERVACIONES_JURIDICAS, en dataset Analitica_UT_Consolidada


In [150]:
df_asignacion_observaciones_juridicas = pd.merge(df_terrenos_asignacion_reconocimiento, df_observaciones_juridicas, left_on='NUMERO_PRE', right_on='codigo', how='inner')
df_asignacion_observaciones_juridicas = pd.DataFrame(df_asignacion_observaciones_juridicas, columns=COLUMNAS_REPORTE_XLS_ASIGNACION_OBSJURIFICAS)
df_asignacion_observaciones_juridicas = df_asignacion_observaciones_juridicas.rename(columns=RENOMBRE_COLUMNAS_REPORTE_XLS_ASIGNACION_OBSJURIDICAS)

In [151]:
ruta_salida_xlsx_asignacion_obsjuridicas = os.path.join(RUTA_XLSX_ASIGNACION_OBSJURIDICAS, NOMBRE_XLSX_ASIGNACION_OBSJURIDICAS)

df_asignacion_observaciones_juridicas.to_excel(ruta_salida_xlsx_asignacion_obsjuridicas, sheet_name='Asignacion_ObsJuridicas')